In [2]:
import pandas as pd
import numpy as np
import time
import datetime
import os

wd = "/Users/finn/Documents/GitHub/RADataHub/prices/data/"

In [3]:
# read all pkls in the folder into a list of dataframes
base_path = "/Users/finn/Documents/GitHub/FM-ds.github.io/EconOb/Prices/scrapes"

dfs = []

""" for path in os.listdir(base_path):
    if path.endswith('.pkl'):
        df = pd.read_pickle(f"{base_path}/{path}")
        dfs.append(df)
df = pd.concat(dfs)
 """

to_read = [
    "2023_07_11_tesco.pkl",
]

for path in to_read:
    df = pd.read_pickle(f"{base_path}/{path}")
    dfs.append(df)
df = pd.concat(dfs)

df['tesco_id'] = df['link'].astype(str).replace("https://www.tesco.com/groceries/en-GB/products/", "", regex=True)

the items csv - this is horribly inneficent:

In [4]:
# build existing list from items.csv
existing_items_df = pd.read_csv(f"{wd}/db_item.csv")
# build existing list from items.csv
custom_ids = {}
for index, row in existing_items_df[['store_id', 'product_id', 'custom_id']].iterrows():
    if not np.isnan(row['custom_id']):
        custom_ids[f"{int(row['store_id'])}_{int(row['product_id'])}"] = int(row['custom_id'])

items_df = df[['name', 'link', 'tesco_id']]
items_df['store_id'] = 1
items_df.rename(columns={'tesco_id': 'product_id', 'name': 'product_title', 'link':'product_url'}, inplace=True)
# get max custom id for tesco
max_existing_id = existing_items_df.query('store_id == 1')['custom_id'].max()
unassigned_ids = list(set([f"{int(x['store_id'])}_{int(x['product_id'])}" for index, x in items_df.iterrows()]) - set(custom_ids.keys()))

# bind custom ids
def id(x):
    if f"{x['store_id']}_{x['product_id']}" in custom_ids.keys():
        return custom_ids[f"{x['store_id']}_{x['product_id']}"]
    else:
        return max_existing_id+unassigned_ids.index(f"{x['store_id']}_{x['product_id']}")


items_df['custom_id'] = items_df.apply(id, axis=1)

items_df['cpi_item_id'] = ''
items_df['brand_id'] = ''
items_df['date_qoute_s'] = ''
items_df['date_quote_e'] = ''
items_df['product_breadcrumb'] = ''
items_df['observations'] = ''

items_df = items_df[['store_id','custom_id', 'cpi_item_id', 'product_id', 'brand_id', 'product_title', 'date_qoute_s', 'date_quote_e', 'observations', 'product_breadcrumb', 'product_url']]



/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_1103/2603662266.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  existing_items_df = pd.read_csv(f"{wd}/db_item.csv")
/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_1103/2603662266.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_df['store_id'] = 1
/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_1103/2603662266.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_df.rename(columns={'tesco_id': 'product_id', '

In [5]:
items_df.drop_duplicates(subset=['store_id', 'custom_id', ], inplace=True)

/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_1103/2384470172.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_df.drop_duplicates(subset=['store_id', 'custom_id', ], inplace=True)


In [6]:
# append headless_csv to db_prices.csv but don't duplicate existing rows

headless_items_csv = items_df.to_csv(header=False, index=False)

with open(f"{wd}db_item.csv", "r") as f:
    existing_entries = f.read().splitlines()
    existing_entries = [line.split(",")[0:3] for line in existing_entries]

lines_to_add = []
with open(f"{wd}db_item.csv", "a") as f:
    #new_lines = headless_csv.splitlines()
    #new_lines = [line for line in new_lines if line.split(",")[0:3] not in existing_entries]
    #f.write("\n".join(new_lines))
    last_char = f.seek(0, os.SEEK_END)
    if(last_char != "\n"):
        f.write("\n")

    f.write(headless_items_csv)


the prices csv:

In [7]:
prices_df = df[['tesco_id','scrape_time', 'price', 'relative_price']]
#remove leading and trailing whitespace from relative_price
prices_df['relative_price'] = prices_df['relative_price'].str.strip()
prices_df['store_id'] = 1
# timestamp column to datetime
prices_df['date'] = pd.to_datetime(prices_df['scrape_time'],unit='s')
# format Date as YYYY-MM-DD
prices_df['date'] = prices_df['date'].dt.strftime('%Y-%m-%d')
prices_df['loyalty_price'] = ''
prices_df['original_price'] = ''

prices_df = prices_df[['store_id', 'tesco_id', 'date', 'price', 'original_price' , 'loyalty_price' , 'relative_price']]
prices_df.rename(columns={'tesco_id':'product_id', 'relative_price': 'unit_price'}, inplace=True)

prices_df = prices_df.drop_duplicates(subset=['store_id', 'product_id', 'date'])

headless_csv = prices_df.to_csv( index=False)
headless_csv = headless_csv[headless_csv.index("\n")+1:]


/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_1103/1478762346.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['relative_price'] = prices_df['relative_price'].str.strip()
/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_1103/1478762346.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['store_id'] = 1
/var/folders/by/095_7x8j04x77vcq4_rvm1_m0000gn/T/ipykernel_1103/1478762346.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [8]:
# append headless_csv to db_prices.csv but don't duplicate existing rows

with open(f"{wd}db_prices.csv", "r") as f:
    existing_entries = f.read().splitlines()
    existing_entries = [line.split(",")[0:3] for line in existing_entries]

lines_to_add = []
with open(f"{wd}db_prices.csv", "a") as f:
    #new_lines = headless_csv.splitlines()
    #new_lines = [line for line in new_lines if line.split(",")[0:3] not in existing_entries]
    #f.write("\n".join(new_lines))
    last_char = f.seek(0, os.SEEK_END)
    print(last_char)
    if(last_char != "\n"):
        f.write("\n")

    f.write(headless_csv)


14623967


the items csv:

In [ ]:
dfs[5][['name', 'link', 'scrape_time', 'prices', 'price', 'relative_price']].to_csv("scrape_sample.csv")

In [ ]:
counts =  df.groupby('link').aggregate({'price': 'count', 'link': 'first'})
df['date'] = pd.to_datetime(df['scrape_time'])
in_all_ids = counts[counts['price'] > 1]['link']
df = df[df['link'].isin(in_all_ids)]


remove duplicate rows from items csv and prices csv:

In [225]:
lines_seen = set() # holds lines already seen
unique_lines = []

for file in [f"{wd}db_prices.csv", f"{wd}db_item.csv"]:
    for line in open(file, "r"):
        if line not in lines_seen: # not a duplicate
            unique_lines.append(line)
            lines_seen.add(line)
    with open(file, "w") as out_file:
        for line in unique_lines:
            out_file.write(line)


In [11]:
df = pd.read_csv(f"{wd}db_prices.csv")
df = df.drop_duplicates(subset=['store_id', 'product_id', 'date'])

In [13]:
df.to_csv(f"{wd}db_prices.csv", index=False)

In [ ]:
# lose the time
df['date'].dt.date


1        1970-01-01
10       1970-01-01
11       1970-01-01
16       1970-01-01
19       1970-01-01
            ...    
26919    1970-01-01
26922    1970-01-01
26923    1970-01-01
26938    1970-01-01
27001    1970-01-01
Name: date, Length: 5845, dtype: object

In [ ]:
pd.concat(dfs)

,name,link,prices,category,price,relative_price,scrape_time,source_page
0,Activia Rhubarb Yogurt 4X115g,https://www.tesco.com/groceries/en-GB/products...,"[£2.50, £0.54/100g]",fresh,2.50,£0.54/100g,1.687320e+09,NaN
1,Danone Actimel Multifruit Drink 12X100g,https://www.tesco.com/groceries/en-GB/products...,"[£4.50, £0.38/100g]",fresh,4.50,£0.38/100g,1.687320e+09,NaN
2,Tesco British Salted Block Butter 250G ++,https://www.tesco.com/groceries/en-GB/products...,"[£1.89, £7.56/kg]",fresh,1.89,£7.56/kg,1.687320e+09,NaN
3,Tesco British Unsalted Butter 250G,https://www.tesco.com/groceries/en-GB/products...,"[£1.89, £7.56/kg]",fresh,1.89,£7.56/kg,1.687320e+09,NaN
4,Tesco Semi Skimmed Milk 1.13L/2 Pints,https://www.tesco.com/groceries/en-GB/products...,"[£1.25, £1.11/litre]",fresh,1.25,£1.11/litre,1.687320e+09,NaN
...,...,...,...,...,...,...,...,...
27018,Purina One Wet Chicken Beef 8X85g,https://www.tesco.com/groceries/en-GB/products...,"[£4.60, £6.76/kg]",NaN,4.60,£6.76/kg,1.688064e+09,None
27019,Sheba Fine Flakes In Gravy Fish Selection 12 X...,https://www.tesco.com/groceries/en-GB/products...,"[£5.50, £5.39/kg]",NaN,5.50,£5.39/kg,1.688064e+09,None
27020,Pedigree Adult Cuts In Jelly Multi Pack 6 X 385G,https://www.tesco.com/groceries/en-GB/products...,"[£7.00, £3.03/kg]",NaN,7.00,£3.03/kg,1.688064e+09,None
27021,Lilys Kitchen Mini Beef Brgrs For Dogs 70G,https://www.tesco.com/groceries/en-GB/products...,"[£3.50, £50.00/kg]",NaN,3.50,£50.00/kg,1.688064e+09,None


In [ ]:
all_files

['failure_log.txt',
 'scrapes',
 'products_starttime.json',
 'products_1687613700.4520671.json',
 'products_fresh_food.pkl',
 'batch.ipynb',
 '.DS_Store',
 'products_cupboard.pkl',
 'tesco_prices_with_embeddings.csv',
 'products_1687876257.791008.json',
 'products_1687608807.858767.json',
 'cpi_with_embeddings.pkl',
 'product_pages.pkl',
 'products_pets.pkl',
 'products_home_ents.pkl',
 'products_household.pkl',
 'products_bakery.pkl',
 'products_1687902012.0459142.json',
 'products_frozen.pkl',
 'geckodriver.log',
 'products_treats_snacks.pkl',
 'products_health_beauty.pkl',
 'products_baby_todler.pkl',
 'products_1687609000.537618.json',
 'products_1687611855.822231.json',
 'products_1687745724.376501.json',
 'products_drinks.pkl',
 'success_log.txt',
 '2023_06_24_tesco.pkl',
 'quants',
 'tesco_prices_with_embeddings.pkl',
 'Selenium_Tesco.ipynb']